In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/martin/Documents/git/GNN/torch_geometric_examples/") 
# custom dataset
from division_v import DivisionV
from division import Division

import os.path as osp

import torch
import torch.nn.functional as F
from sklearn.metrics import r2_score

from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv

In [2]:
# constants

PATH = '/home/martin/graphs/division'

In [3]:

train_dataset   = DivisionV(PATH, split='train')
val_dataset     = DivisionV(PATH, split='val')
test_dataset    = DivisionV(PATH, split='test')

In [4]:
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=12, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=True)

In [5]:
for data in train_loader:
    print(data)

DataBatch(x=[121794, 1], edge_index=[2, 219733], edge_attr=[219733, 1], y=[121794, 1], num_node_features=[12], num_nodes=121794, batch=[121794], ptr=[13])
DataBatch(x=[124726, 1], edge_index=[2, 228674], edge_attr=[228674, 1], y=[124726, 1], num_node_features=[12], num_nodes=124726, batch=[124726], ptr=[13])
DataBatch(x=[124370, 1], edge_index=[2, 225212], edge_attr=[225212, 1], y=[124370, 1], num_node_features=[12], num_nodes=124370, batch=[124370], ptr=[13])
DataBatch(x=[122928, 1], edge_index=[2, 231317], edge_attr=[231317, 1], y=[122928, 1], num_node_features=[12], num_nodes=122928, batch=[122928], ptr=[13])
DataBatch(x=[124281, 1], edge_index=[2, 224064], edge_attr=[224064, 1], y=[124281, 1], num_node_features=[12], num_nodes=124281, batch=[124281], ptr=[13])
DataBatch(x=[113698, 1], edge_index=[2, 206621], edge_attr=[206621, 1], y=[113698, 1], num_node_features=[12], num_nodes=113698, batch=[113698], ptr=[13])
DataBatch(x=[80713, 1], edge_index=[2, 142173], edge_attr=[142173, 1],

In [6]:
class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GATConv(train_dataset.num_features, 256, heads=4, edge_dim=1)
        self.lin1 = torch.nn.Linear(train_dataset.num_features, 4 * 256)
        self.conv2 = GATConv(4 * 256, 256, heads=4, edge_dim=1)
        self.lin2 = torch.nn.Linear(4 * 256, 4 * 256)
        self.conv3 = GATConv(4 * 256, 1, heads=6,
                             concat=False, edge_dim=1)
        self.lin3 = torch.nn.Linear(4 * 256, 1)

    def forward(self, x, edge_index, edge_attr):
        x = F.relu(self.conv1(x, edge_index, edge_attr) + self.lin1(x))
        x = F.relu(self.conv2(x, edge_index, edge_attr) + self.lin2(x))
        x = self.conv3(x, edge_index) + self.lin3(x)
        return x

In [7]:
from torchsummary import summary

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
loss_op = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [9]:
su = summary(model)

Layer (type:depth-idx)                   Param #
├─GATConv: 1-1                           --
|    └─Linear: 2-1                       1,024
|    └─Linear: 2-2                       1,024
├─Linear: 1-2                            2,048
├─GATConv: 1-3                           --
|    └─Linear: 2-3                       1,048,576
|    └─Linear: 2-4                       1,024
├─Linear: 1-4                            1,049,600
├─GATConv: 1-5                           --
|    └─Linear: 2-5                       6,144
|    └─Linear: 2-6                       6
├─Linear: 1-6                            1,025
Total params: 2,110,471
Trainable params: 2,110,471
Non-trainable params: 0


In [10]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:

        
        data = data.to(device)
        optimizer.zero_grad()
        rt = model(data.x, data.edge_index, data.edge_attr)
        loss = loss_op(rt, data.y)
        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)

In [11]:
for epoch in range(1, 101):
    loss = train()
    
    # val_f1 = test(val_loader)
    # test_f1 = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    #       f'Test: {test_f1:.4f}')

Epoch: 001, Loss: 171.7086
Epoch: 002, Loss: 5.3835
Epoch: 003, Loss: 0.4167
Epoch: 004, Loss: 0.0976
Epoch: 005, Loss: 0.0056
Epoch: 006, Loss: 0.0050
Epoch: 007, Loss: 0.0017
Epoch: 008, Loss: 0.0017
Epoch: 009, Loss: 0.0011
Epoch: 010, Loss: 0.0007
Epoch: 011, Loss: 0.0006
Epoch: 012, Loss: 0.0003
Epoch: 013, Loss: 0.0001
Epoch: 014, Loss: 0.0001
Epoch: 015, Loss: 0.0001
Epoch: 016, Loss: 0.0000
Epoch: 017, Loss: 0.0000
Epoch: 018, Loss: 0.0000
Epoch: 019, Loss: 0.0000
Epoch: 020, Loss: 0.0000
Epoch: 021, Loss: 0.0000
Epoch: 022, Loss: 0.0000
Epoch: 023, Loss: 0.0000
Epoch: 024, Loss: 0.0000
Epoch: 025, Loss: 0.0000
Epoch: 026, Loss: 0.0000
Epoch: 027, Loss: 0.0000
Epoch: 028, Loss: 0.0000
Epoch: 029, Loss: 0.0000
Epoch: 030, Loss: 0.0000
Epoch: 031, Loss: 0.0000
Epoch: 032, Loss: 0.0000
Epoch: 033, Loss: 0.0000
Epoch: 034, Loss: 0.0000
Epoch: 035, Loss: 0.0000
Epoch: 036, Loss: 0.0000
Epoch: 037, Loss: 0.0000
Epoch: 038, Loss: 0.0000
Epoch: 039, Loss: 0.0000
Epoch: 040, Loss: 0.000

In [12]:
ID = 2
DS = 0
model.eval()
out = model(test_dataset.data.x.to(device), test_dataset.data.edge_index.to(device), test_dataset.datedge_attr.to(device))
print(test_dataset[DS][0].x.numpy()[ID])
out.cpu().detach().numpy()[ID][0]


KeyError: 0

In [ ]:
loss